In [1]:
from parsing_on_batch import process_html_and_extract_data
from extract_table import create_dataframes
import json
import pandas as pd
import numpy as np 


In [2]:
header_df,body_df=create_dataframes()
llm_output_list= process_html_and_extract_data()

In [3]:
print(len(header_df))

89


In [4]:
header_df[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,,Head,,,,,,,
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.


In [5]:
body_df[3].head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,2-56,,,,,,,,,,,
1,Black-Oxide Alloy Steel,,,,,,,,,,,
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51
5,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12
6,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93
7,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08
8,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24
9,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22


In [6]:
print(llm_output_list[3])

{
  "thread_size": [
    "2-56"
  ],
  "material_surface": [
    "Black-Oxide Alloy Steel",
    "Zinc-Plated Alloy Steel"
  ]
}


In [7]:
head_row_len=header_df[3].shape[0]

In [8]:
header_df[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,,Head,,,,,,,
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.


In [9]:
no_cols=header_df[3].shape[1]

In [10]:
parsed_dict = json.loads(llm_output_list[3])
keys = list(parsed_dict.keys())


In [11]:
head_df=header_df[3]
for i in range(len(keys)):
    new_column_name = f"{no_cols + 1}" 
    head_df[new_column_name] = np.nan  
    head_df.loc[head_row_len-1, new_column_name] = keys[i]  
    no_cols += 1 

header_df[0]=head_df
    

/tmp/ipykernel_34205/1206480191.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'thread_size' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  head_df.loc[head_row_len-1, new_column_name] = keys[i]
/tmp/ipykernel_34205/1206480191.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'material_surface' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  head_df.loc[head_row_len-1, new_column_name] = keys[i]


In [12]:
header_df[3]

,0,1,2,3,4,5,6,7,8,9,10,11,13,14
0,,,,,Head,,,,,,,,NaN,NaN
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface


In [13]:
header_df[3].shape

(2, 14)

In [14]:
body_df[3].head()


,0,1,2,3,4,5,6,7,8,9,10,11
0,2-56,,,,,,,,,,,
1,Black-Oxide Alloy Steel,,,,,,,,,,,
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51


In [15]:
merged_df = pd.concat([header_df[3].iloc[[-1]], body_df[3]], ignore_index=True)
merged_df.head()
# Step 1: Set column headers using the SECOND ROW (index 1)
merged_df.columns = merged_df.iloc[0]  # <-- Fix: Use iloc[1], not iloc[0]

# Step 2: Drop ONLY the first two rows (numeric row 0 and header row 1)
merged_df = merged_df.drop([0])

# Step 3: Reset index to avoid gaps
merged_df = merged_df.reset_index(drop=True)

# Step 4: Clean column names (strip whitespace)
merged_df.columns = merged_df.columns.str.strip()

In [16]:
merged_df.head(30)

,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
0,2-56,,,,,,,,,,,,NaN,NaN
1,Black-Oxide Alloy Steel,,,,,,,,,,,,NaN,NaN
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,NaN,NaN
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,NaN,NaN
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,NaN,NaN
5,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,NaN,NaN
6,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,NaN,NaN
7,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,NaN,NaN
8,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24,NaN,NaN
9,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22,NaN,NaN


In [17]:
merged_df.shape

(33, 14)

In [18]:
"""def map_dict_data(data_dict, dataframe):
    # Step 1: Extract the thread_size list from the dictionary
    thread_sizes = data_dict.get('thread_size', [])
    
    # Initialize the thread_size column with None
    dataframe['thread_size'] = None
    current_thread = None
    
    # Get the name of the first column
    first_col_name = dataframe.columns[0]
    
    # Step 2 & 3: Iterate through each row to assign thread_size based on the algorithm
    for index, row in dataframe.iterrows():
        first_col_value = row[first_col_name]
        if first_col_value in thread_sizes:
            current_thread = first_col_value
        dataframe.at[index, 'thread_size'] = current_thread
    
    return dataframe"""
def assign_thread_size(data_dict, dataframe):
    # Step 1: Extract the thread_size list from the dictionary
    thread_sizes = data_dict.get('thread_size', [])
    
    # Initialize the thread_size column with None
    dataframe['thread_size'] = None
    current_thread = None
    
    # Get the name of the first column
    first_col_name = dataframe.columns[0]
    
    # Step 2 & 3: Iterate through each row to assign thread_size
    for index, row in dataframe.iterrows():
        first_col_value = row[first_col_name]
        if first_col_value in thread_sizes:
            current_thread = first_col_value
        dataframe.at[index, 'thread_size'] = current_thread
    
    # Step 4: Remove rows where the first column value matches any thread_size value
    dataframe = dataframe[~dataframe[first_col_name].isin(thread_sizes)].reset_index(drop=True)
    
    return dataframe

In [19]:
assign_thread_size_df = assign_thread_size(parsed_dict, merged_df)

In [20]:
def assign_material_surface(data_dict, dataframe):
   
    material_surfaces = data_dict.get('material_surface', [])
    
  
    dataframe['material_surface'] = None
    current_thread = None
    
   
    first_col_name = dataframe.columns[0]
    
  
    for index, row in dataframe.iterrows():
        first_col_value = row[first_col_name]
        if first_col_value in material_surfaces:
            current_thread = first_col_value
        dataframe.at[index, 'material_surface'] = current_thread
    
   
    dataframe = dataframe[~dataframe[first_col_name].isin(material_surfaces)].reset_index(drop=True)
    
    return dataframe

In [21]:
assign_material_surface_df = assign_material_surface(parsed_dict, assign_thread_size_df)

In [22]:
assign_material_surface_df.head(35)

,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
0,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,2-56,Black-Oxide Alloy Steel
1,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,2-56,Black-Oxide Alloy Steel
2,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,2-56,Black-Oxide Alloy Steel
3,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,2-56,Black-Oxide Alloy Steel
4,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,2-56,Black-Oxide Alloy Steel
5,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,2-56,Black-Oxide Alloy Steel
6,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24,2-56,Black-Oxide Alloy Steel
7,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22,2-56,Black-Oxide Alloy Steel
8,"7/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A060,14.06,2-56,Black-Oxide Alloy Steel
9,"1/2""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A081,9.36,2-56,Black-Oxide Alloy Steel


In [23]:
"""header_df[3].columns = header_df[3].iloc[0]   
header_df[3] = header_df[3].reset_index(drop=True) 


assign_material_surface_df.columns = header_df[3].columns"""

'header_df[3].columns = header_df[3].iloc[0]   \nheader_df[3] = header_df[3].reset_index(drop=True) \n\n\nassign_material_surface_df.columns = header_df[3].columns'

In [24]:
"""header_row = header_df[3].iloc[:1]  # Extracts only the first row

# Merge the header row with assign_material_surface_df
merged_df = pd.concat([header_row, assign_material_surface_df], axis=0, ignore_index=True)"""
merged_df = pd.concat([header_df[3], assign_material_surface_df], axis=0, ignore_index=True)

In [25]:
header_columns = header_df[3].columns.tolist()
assign_columns = assign_material_surface_df.columns.tolist()
print("Columns in A:", header_columns)
print("Columns in B:", assign_columns)

Columns in A: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, '13', '14']
Columns in B: ['Lg.', 'Threading', 'Min.Thread Lg.', 'Thread Spacing', 'Dia.', 'Ht.', 'DriveSize', 'TensileStrength, psi', 'Specifications Met', 'Pkg.Qty.', '', 'Pkg.', 'thread_size', 'material_surface']


In [26]:
# Create a dictionary to map header_df columns to assign_material_surface_df columns
column_mapping = {header_col: assign_col for header_col, assign_col in zip(header_columns, assign_columns)}

# Rename columns of header_df to match assign_material_surface_df
header_df_renamed = header_df[3].rename(columns=column_mapping)

# Merge the two DataFrames vertically
merged_df = pd.concat([header_df_renamed, assign_material_surface_df], axis=0, ignore_index=True)

In [27]:
#merged_df = merged_df.drop(index=0).reset_index(drop=True)

In [28]:
merged_df.head(50)

,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
0,,,,,Head,,,,,,,,NaN,NaN
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,2-56,Black-Oxide Alloy Steel
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,2-56,Black-Oxide Alloy Steel
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,2-56,Black-Oxide Alloy Steel
5,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,2-56,Black-Oxide Alloy Steel
6,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,2-56,Black-Oxide Alloy Steel
7,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,2-56,Black-Oxide Alloy Steel
8,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24,2-56,Black-Oxide Alloy Steel
9,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22,2-56,Black-Oxide Alloy Steel


In [29]:
merged_df.shape

(32, 14)

In [ ]:
def save_dataframes_to_excel(merged_df, filename="./mcmaster_excel/output8.xlsx"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    if not merged_df:
        print("No data to save")
        return

    try:
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            for idx, df in enumerate(merged_df, 1):
                df.to_excel(writer, sheet_name=f"Sheet{idx}", index=False)
        print(f"Successfully saved to {filename}")
    except Exception as e:
        print(f"Save error: {e}")

NameError: name 'qqq' is not defined

In [ ]:
body_sliced = assign_material_surface_df.reset_index(drop=True) 

In [ ]:
body_sliced.head(20)

,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
0,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,2-56,Black-Oxide Alloy Steel
1,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,2-56,Black-Oxide Alloy Steel
2,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,2-56,Black-Oxide Alloy Steel
3,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,2-56,Black-Oxide Alloy Steel
4,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,2-56,Black-Oxide Alloy Steel
5,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,2-56,Black-Oxide Alloy Steel
6,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24,2-56,Black-Oxide Alloy Steel
7,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22,2-56,Black-Oxide Alloy Steel
8,"7/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A060,14.06,2-56,Black-Oxide Alloy Steel
9,"1/2""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A081,9.36,2-56,Black-Oxide Alloy Steel


In [ ]:
merged_df = pd.concat([header_sliced, body_sliced], axis=1)

In [ ]:
merged_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
0,,,,,Head,,,,,,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,2-56,Black-Oxide Alloy Steel
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,2-56,Black-Oxide Alloy Steel
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,2-56,Black-Oxide Alloy Steel
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,2-56,Black-Oxide Alloy Steel
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,2-56,Black-Oxide Alloy Steel
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,2-56,Black-Oxide Alloy Steel
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24,2-56,Black-Oxide Alloy Steel
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22,2-56,Black-Oxide Alloy Steel
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A060,14.06,2-56,Black-Oxide Alloy Steel
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A081,9.36,2-56,Black-Oxide Alloy Steel
